#### Import

In [1]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *

In [2]:
import urx
import math3d as m3d
ur = urx.Robot("192.168.1.10")
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

<Transform:
<Orientation: 
array([[-1.13445515e-04,  8.65996364e-01, -5.00050283e-01],
       [ 9.99999991e-01,  1.33176672e-04,  3.76962167e-06],
       [ 6.98595113e-05, -5.00050278e-01, -8.65996371e-01]])>
<Vector: (-0.51000, 0.39997, 0.59058)>
>


In [3]:
from ddh import DDGripper
from scooping_primitives import HighSpeedScooping
gripper = DDGripper("ddh_scooping")
scoop = HighSpeedScooping(ur, gripper, "HSS")

#### scooping test

In [ ]:
scoop.initialize_gripper_pose((-0.65,0.4,180))
time.sleep(0.5)
a2_init = (gripper.left_a2 + gripper.right_a2)/2 

In [ ]:
# smack and scoop
smack_vel = 100
smack_acc = 0.3
stop_smack_acc = 7
reverse_vel = 250
reverse_acc = 1.0 #0.5
stop_acc = 0.5
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)
# robot detect collision, then stop, close gripper, and go up
while 1:
    a2_cur = (gripper.left_a2 + gripper.right_a2)/2
    #print(a2_cur)
    if a2_cur - a2_init > 0.3:
        print ("Collision detected!")
        gripper.set_left_a1_phi(-0, 20)
        gripper.set_right_a1_phi(0, -20)
        ur.stopl(stop_smack_acc)
        ur.speedl([0,0,reverse_vel,0,0,0],reverse_acc,5)
        time.sleep(0.25)
        ur.stopl(stop_acc)
        break

In [ ]:
ur.stopl()